1. Importing required libraries - in this case only pandas was enough

In [1]:
import pandas as pd                 #we deal with pandas for the table part
from itertools import combinations  #to deal with subsets

2. Importing data and getting required info

In [2]:
data = [
        ['T100',['I1','I2','I3']],
        ['T200',['I2','I3','I4']],
        ['T300',['I4','I5']],
        ['T400',['I1','I2','I4']],
        ['T500',['I1','I2','I3','I5']],
        ['T600',['I1','I2','I3','I4']]
        ]
print('Input data:')
data

Input data:


[['T100', ['I1', 'I2', 'I3']],
 ['T200', ['I2', 'I3', 'I4']],
 ['T300', ['I4', 'I5']],
 ['T400', ['I1', 'I2', 'I4']],
 ['T500', ['I1', 'I2', 'I3', 'I5']],
 ['T600', ['I1', 'I2', 'I3', 'I4']]]

We have 6 transaction it is very simple but helps in understanding the data and how the algorithm works

In [3]:
#get all the unique items from the data
uniq_item=[]

for i in data:
  for j in i[1]:
    if j not in uniq_item:
      uniq_item.append(j)

uniq_item = sorted(uniq_item)
uniq_item

['I1', 'I2', 'I3', 'I4', 'I5']

For this learning lets use min support as 50% and min confident as 60%

In [4]:
support = 0.5

support_val = support*len(data)
support_val

3.0

3. Apririo algorithm has 2 important steps to find the frequent items
    1. Joining - We find the particular item or set of items of the data and get support of those
    2. Pruning - Based on the support threshold we will remove the subset from our final table

We iterate these steps will we find the most frequent subsets.... Let's start

---
1st iteration: Unique items

In [5]:
#Joining

items = []
for i in data:
  for j in i[1]:
    items.append(j)

calc_it1 = pd.Series(items)
calc_it1 = calc_it1.value_counts().sort_index()
print(f'Calculated: \n{calc_it1}')

#pruning
items = []
count = []
for num,i in enumerate(calc_it1 >= support_val): #filtering the table with support threshold
  if i:
    items.append(calc_it1.index[num])
    count.append(calc_it1[num])

it1 = pd.DataFrame(data={'Items': items, 'Count': count})
print(f'Pruned: \n{it1}')

Calculated: 
I1    4
I2    5
I3    4
I4    4
I5    2
dtype: int64
Pruned: 
  Items  Count
0    I1      4
1    I2      5
2    I3      4
3    I4      4


---
2nd iteration: Creating pairs

In [6]:
#Joining

items = []
for num,i in enumerate(it1.Items):
  for j in it1.Items[num:]:
    # num = 0
    for k in range(0, len(data)):
      if (i != j) and (j in data[k][1]) and (i in data[k][1]):
          # num = num+1
          # print(data[k][1], i, j, num)
          items.append([i, j])
    # print(i, j, num)

calc_it2 = pd.Series(items)
calc_it2 = calc_it2.value_counts().sort_index()
print(f'Calculated: \n{calc_it2}')

#pruning
items = []
count = []
# filtering the table with support threshold
for num, i in enumerate(calc_it2 >= support_val):
  if i:
    items.append(calc_it2.index[num])
    count.append(calc_it2[num])

it2 = pd.DataFrame(data= {'Items': items, 'Count': count})
print(f'Pruned: \n{it2}')


Calculated: 
[I1, I2]    4
[I1, I3]    3
[I1, I4]    2
[I2, I3]    4
[I2, I4]    3
[I3, I4]    2
dtype: int64
Pruned: 
      Items  Count
0  [I1, I2]      4
1  [I1, I3]      3
2  [I2, I3]      4
3  [I2, I4]      3


---
3nd iteration: Creating pairs of 3

In [7]:
#Joining

items = []
for num1, i in enumerate(it1.Items):
  for num2, j in enumerate(it1.Items[num1:]):
    for k in it1.Items[num2:]:
        # num = 0
        for m in range(0, len(data)):
            if ((i != j) and (i != k) and (j != k)) and (i in data[m][1]) and (j in data[m][1]) and (k in data[m][1]):
                # num = num+1
                # print(data[m][1], i, j, k, num)
                items.append([i, j, k])

calc_it3 = pd.Series(items)
calc_it3 = calc_it3.value_counts().sort_index()
print(f'Calculated: \n{calc_it3}')

#pruning
items = []
count = []
for num, i in enumerate(calc_it3 >= support_val): #filtering the table with support threshold
  if i:
    items.append(calc_it3.index[num])
    count.append(calc_it3[num])


it3 = pd.DataFrame(data={'Items': items, 'Count': count})

print(f'Pruned: \n{it3}')


Calculated: 
[I1, I2, I3]    3
[I1, I2, I4]    2
[I1, I3, I4]    1
[I2, I3, I4]    2
[I2, I4, I3]    2
[I3, I4, I2]    2
dtype: int64
Pruned: 
          Items  Count
0  [I1, I2, I3]      3


---
4nd iteration: Creating pairs of 4


Before we do this if we see the actual data we can notice that there are only 2 transaction with 4 items.

        ['T500',['I1','I2','I3','I5']]
        ['T600',['I1','I2','I3','I4']]

support of each will be just 1/6 = 0.16 i.e 16% while our support threshold is 50% so we can safely ignore this and apply Generate Association Rules and see the results

4. Association Rules

In [8]:
final_table = pd.concat([it1, it2, it3], axis=0, ignore_index=True)
final_table

,Items,Count
0,I1,4
1,I2,5
2,I3,4
3,I4,4
4,"[I1, I2]",4
5,"[I1, I3]",3
6,"[I2, I3]",4
7,"[I2, I4]",3
8,"[I1, I2, I3]",3


In [9]:
# function to generate all the sub lists of the list
def sub_lists(l):
    # initializing empty list
    comb = []

    #Iterating till length of list
    for i in range(len(l)+1):
        # Generating sub list
        sub_list = [list(j) for j in combinations(l, i)]
        for k in sub_list:
            if len(k) > 0: #remove empty list element we get
                comb += [k]
                
    # Returning list
    return comb

for i in it3.Items:
    sub_set = sub_lists(i)

print('Sublist created: \n',sub_set)


Sublist created: 
 [['I1'], ['I2'], ['I3'], ['I1', 'I2'], ['I1', 'I3'], ['I2', 'I3'], ['I1', 'I2', 'I3']]


In [10]:
#function to calculate support for a items or pairs or items
def support_calc(item): 
    val = final_table[final_table["Items"].isin([item])].Count/len(data) #getting the count/support of items like I2 or [I3, I4] 
    return val.values


#note when passing done use [] for single item but use it for pairs 


In [11]:
#lets calculate confidence 

#confidence of people who buys (I1, I2)  will buy I3:   (I1, I2) -> I3
conf_I1_I2 = support_calc(['I1', 'I2', 'I3']) / support_calc(['I1', 'I2'])
print('confidence of (I1, I2) -> I3: ', int(conf_I1_I2*100),'%')

#confidence of people who buys (I1, I3)  will buy I2:   (I1, I3) -> I2
conf_I1_I3 = support_calc(['I1', 'I2', 'I3']) / support_calc(['I1', 'I3'])
print('confidence of (I1, I3) -> I2: ', int(conf_I1_I3*100),'%')

#confidence of people who buys (I2, I3)  will buy I1:   (I2, I3) -> I1
conf_I2_I3 = support_calc(['I1', 'I2', 'I3']) / support_calc(['I2', 'I3'])
print('confidence of (I2, I3) -> I1: ', int(conf_I2_I3*100),'%')

#confidence of people who buys I1 will buy (I2, I3):   I1 -> (I2, I3)
conf_I1 = support_calc(['I1', 'I2', 'I3']) / support_calc('I1')
print('confidence of I1 -> (I2, I3): ', int(conf_I1*100),'%')

#confidence of people who buys I2 will aso buy (I1, I3):   I2 -> (I1, I3)
conf_I2 = support_calc(['I1', 'I2', 'I3']) / support_calc('I2')
print('confidence of I2 -> (I1, I3): ', int(conf_I2*100),'%')

#confidence of people who buys I3 will buy (I1, I2):   I3 -> (I1, I2)
conf_I3 = support_calc(['I1', 'I2', 'I3']) / support_calc('I3')
print('confidence of I3 -> (I1, I2): ', int(conf_I3*100),'%')

confidence of (I1, I2) -> I3:  75 %
confidence of (I1, I3) -> I2:  100 %
confidence of (I2, I3) -> I1:  75 %
confidence of I1 -> (I2, I3):  75 %
confidence of I2 -> (I1, I3):  60 %
confidence of I3 -> (I1, I2):  75 %


From the result we can see all are equal or above the min confidence threshold so we come to the conclusing that all these are applicable based on the thresholds provided by stakeholders/marketing team

Please go back to the README.md file for more information